In [45]:
import numpy as np
import pandas as pd
import re
import nltk
import contractions
from nltk.corpus import stopwords

file_path = r"C:\Users\ryan_\OneDrive\Desktop\Bipolar\Data\bipolar_dataset.csv"

# Define column names based on the first line of the file
with open(file_path, "r") as f:
    columns = f.readline().strip().split(";")

# Read the file into a DataFrame
df = pd.read_csv(file_path, sep=";", skiprows=1, header=None, encoding='latin1')

# Assign column names
df.columns = columns

# Display the DataFrame
pd.set_option('display.max_columns', None)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1692 entries, 0 to 1691
Columns: 350 entries, subreddit to tfidf_year
dtypes: int64(77), object(273)
memory usage: 4.5+ MB


In [27]:
df.head()

,subreddit,author,date,post,automated_readability_index,coleman_liau_index,flesch_kincaid_grade_level,flesch_reading_ease,gulpease_index,gunning_fog_index,lix,smog_index,wiener_sachtextformel,n_chars,n_long_words,n_monosyllable_words,n_polysyllable_words,n_sents,n_syllables,n_unique_words,n_words,sent_neg,sent_neu,sent_pos,sent_compound,economic_stress_total,isolation_total,substance_use_total,guns_total,domestic_stress_total,suicidality_total,punctuation,liwc_1st_pers,liwc_2nd_pers,liwc_3rd_pers,liwc_achievement,liwc_adverbs,liwc_affective_processes,liwc_anger,liwc_anxiety,liwc_articles_article,liwc_assent,liwc_auxiliary_verbs,liwc_biological,liwc_body,liwc_causation,liwc_certainty,liwc_cognitive,liwc_common_verbs,liwc_conjunctions,liwc_death,liwc_discrepancy,liwc_exclusive,liwc_family,liwc_feel,liwc_fillers,liwc_friends,liwc_future_tense,liwc_health,liwc_hear,liwc_home,liwc_humans,liwc_impersonal_pronouns,liwc_inclusive,liwc_ingestion,liwc_inhibition,liwc_insight,liwc_leisure,liwc_money,liwc_motion,liwc_negations,liwc_negative_emotion,liwc_nonfluencies,liwc_numbers,liwc_past_tense,liwc_perceptual_processes,liwc_personal_pronouns,liwc_positive_emotion,liwc_prepositions,liwc_present_tense,liwc_quantifiers,liwc_relativity,liwc_religion,liwc_sadness,liwc_see,liwc_sexual,liwc_social_processes,liwc_space,liwc_swear_words,liwc_tentative,liwc_time,liwc_total_functional,liwc_total_pronouns,liwc_work,tfidf_abl,tfidf_abus,tfidf_actual,tfidf_addict,tfidf_adhd,tfidf_advic,tfidf_ago,tfidf_alcohol,tfidf_almost,tfidf_alon,tfidf_alreadi,tfidf_also,tfidf_alway,tfidf_amp,tfidf_amp x200b,tfidf_ani,tfidf_anoth,tfidf_anxieti,tfidf_anxious,tfidf_anymor,tfidf_anyon,tfidf_anyon els,tfidf_anyth,tfidf_around,tfidf_ask,tfidf_attack,tfidf_away,tfidf_back,tfidf_bad,tfidf_becaus,tfidf_becom,tfidf_befor,tfidf_believ,tfidf_best,tfidf_better,tfidf_bit,tfidf_bodi,tfidf_bpd,tfidf_brain,tfidf_call,tfidf_came,tfidf_care,tfidf_caus,tfidf_chang,tfidf_come,tfidf_complet,tfidf_constant,tfidf_control,tfidf_could,tfidf_coupl,tfidf_cri,tfidf_day,tfidf_deal,tfidf_depress,tfidf_diagnos,tfidf_die,tfidf_differ,tfidf_disord,tfidf_doctor,tfidf_doe,tfidf_done,tfidf_dont,tfidf_drink,tfidf_drug,tfidf_eat,tfidf_els,tfidf_emot,tfidf_end,tfidf_enough,tfidf_etc,tfidf_even,tfidf_ever,tfidf_everi,tfidf_everyon,tfidf_everyth,tfidf_experi,tfidf_famili,tfidf_fear,tfidf_feel,tfidf_feel like,tfidf_felt,tfidf_final,tfidf_find,tfidf_first,tfidf_food,tfidf_found,tfidf_friend,tfidf_fuck,tfidf_get,tfidf_give,tfidf_go,tfidf_good,tfidf_got,tfidf_great,tfidf_guess,tfidf_guy,tfidf_happen,tfidf_happi,tfidf_hard,tfidf_hate,tfidf_head,tfidf_health,tfidf_hear,tfidf_heart,tfidf_help,tfidf_high,tfidf_home,tfidf_hope,tfidf_hour,tfidf_hous,tfidf_hurt,tfidf_idea,tfidf_im,tfidf_issu,tfidf_job,tfidf_keep,tfidf_kill,tfidf_kind,tfidf_know,tfidf_last,tfidf_late,tfidf_leav,tfidf_left,tfidf_let,tfidf_life,tfidf_like,tfidf_littl,tfidf_live,tfidf_long,tfidf_look,tfidf_lose,tfidf_lost,tfidf_lot,tfidf_love,tfidf_made,tfidf_make,tfidf_mani,tfidf_mayb,tfidf_mean,tfidf_med,tfidf_medic,tfidf_mental,tfidf_might,tfidf_mind,tfidf_mom,tfidf_month,tfidf_move,tfidf_much,tfidf_need,tfidf_never,tfidf_new,tfidf_next,tfidf_night,tfidf_normal,tfidf_noth,tfidf_notic,tfidf_old,tfidf_onc,tfidf_one,tfidf_onli,tfidf_pain,tfidf_panic,tfidf_parent,tfidf_part,tfidf_past,tfidf_peopl,tfidf_person,tfidf_place,tfidf_pleas,tfidf_point,tfidf_possibl,tfidf_post,tfidf_pretti,tfidf_probabl,tfidf_problem,tfidf_ptsd,tfidf_put,tfidf_question,tfidf_quit,tfidf_read,tfidf_real,tfidf_realli,tfidf_reason,tfidf_recent,tfidf_relationship,tfidf_rememb,tfidf_right,tfidf_said,tfidf_say,tfidf_scare,tfidf_school,tfidf_see,tfidf_seem,tfidf_self,tfidf_sever,tfidf_shit,tfidf_sinc,tfidf_situat,tfidf_sleep,tfidf_social,tfidf_someon,tfidf_someth,tfidf_sometim,tfidf_sorri,tfidf_start,tfidf_stay,tfidf_still,tfidf_stop,tfidf_stress,tfidf_struggl,tfidf_stuff,tfidf_suicid,tfidf_support,tfidf_sure,tfidf_symptom,tfidf_take,tfidf_talk,tfidf_tell,tfidf_thank,tfidf_therapi,tfidf

unable to stop talking to myself any ideasexperiences hey all i know this has come up on here before but i find that particularly in times of anxiety i cannot stop myself from saying things out loud they arent even my thoughts necessarily theyre just parts of conversations random sentences etc i sound insane im at home for the holidays and its starting to confuse my family who dont know about my diagnosis what have your experiences with this been has anything in particular helped you 


In [46]:
'''Cleaning Text'''
def expand_contractions(text):
    # Define your custom_contractions dictionary here
    custom_contractions = {
        "ain't": "am not are not is not has not have not",
        "aren't": "are not",
        "can't": "cannot",
        "could've": "could have",
        "couldn't": "could not",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he had he would",
        "he'll": "he shall he will",
        "he's": "he has he is",
        "I'd": "I had I would",
        "I'll": "I shall I will",
        "I'm": "I am",
        "I've": "I have",
        "isn't": "is not",
        "it'd": "it had it would",
        "it'll": "it shall it will",
        "it's": "it has it is",
        "let's": "let us",
        "mightn't": "might not",
        "mustn't": "must not",
        "shan't": "shall not",
        "she'd": "she had she would",
        "she'll": "she shall she will",
        "she's": "she has she is",
        "should've": "should have",
        "shouldn't": "should not",
        "that's": "that has that is",
        "there's": "there has there is",
        "they'd": "they had they would",
        "they'll": "they shall they will",
        "they're": "they are",
        "they've": "they have",
        "wasn't": "was not",
        "we'd": "we had we would",
        "we'll": "we shall we will",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what shall what will",
        "what're": "what are",
        "what's": "what has what is",
        "what've": "what have",
        "where's": "where has where is",
        "who'd": "who had who would",
        "who'll": "who shall who will",
        "who're": "who are",
        "who's": "who has who is",
        "who've": "who have",
        "won't": "will not",
        "wouldn't": "would not",
        "you'd": "you had you would",
        "you'll": "you shall you will",
        "you're": "you are",
        "you've": "you have",
    
    }
    # Replace contractions with their expansions
    for contraction, expansion in custom_contractions.items():
        text = text.replace(contraction, expansion)
    return text

df['post'] = df.iloc[ :,3].apply(expand_contractions)


def clean_text(text):
    text = text.replace("/", " or ")
    text = re.sub(r'[^a-zA-Z\s]','', text) #removes punctuation and special characters
    text = text.lower()#makes everything lower case
    return text
df['post'] = df.iloc[ :,3].apply(clean_text)
df.head()


,subreddit,author,date,post,automated_readability_index,coleman_liau_index,flesch_kincaid_grade_level,flesch_reading_ease,gulpease_index,gunning_fog_index,lix,smog_index,wiener_sachtextformel,n_chars,n_long_words,n_monosyllable_words,n_polysyllable_words,n_sents,n_syllables,n_unique_words,n_words,sent_neg,sent_neu,sent_pos,sent_compound,economic_stress_total,isolation_total,substance_use_total,guns_total,domestic_stress_total,suicidality_total,punctuation,liwc_1st_pers,liwc_2nd_pers,liwc_3rd_pers,liwc_achievement,liwc_adverbs,liwc_affective_processes,liwc_anger,liwc_anxiety,liwc_articles_article,liwc_assent,liwc_auxiliary_verbs,liwc_biological,liwc_body,liwc_causation,liwc_certainty,liwc_cognitive,liwc_common_verbs,liwc_conjunctions,liwc_death,liwc_discrepancy,liwc_exclusive,liwc_family,liwc_feel,liwc_fillers,liwc_friends,liwc_future_tense,liwc_health,liwc_hear,liwc_home,liwc_humans,liwc_impersonal_pronouns,liwc_inclusive,liwc_ingestion,liwc_inhibition,liwc_insight,liwc_leisure,liwc_money,liwc_motion,liwc_negations,liwc_negative_emotion,liwc_nonfluencies,liwc_numbers,liwc_past_tense,liwc_perceptual_processes,liwc_personal_pronouns,liwc_positive_emotion,liwc_prepositions,liwc_present_tense,liwc_quantifiers,liwc_relativity,liwc_religion,liwc_sadness,liwc_see,liwc_sexual,liwc_social_processes,liwc_space,liwc_swear_words,liwc_tentative,liwc_time,liwc_total_functional,liwc_total_pronouns,liwc_work,tfidf_abl,tfidf_abus,tfidf_actual,tfidf_addict,tfidf_adhd,tfidf_advic,tfidf_ago,tfidf_alcohol,tfidf_almost,tfidf_alon,tfidf_alreadi,tfidf_also,tfidf_alway,tfidf_amp,tfidf_amp x200b,tfidf_ani,tfidf_anoth,tfidf_anxieti,tfidf_anxious,tfidf_anymor,tfidf_anyon,tfidf_anyon els,tfidf_anyth,tfidf_around,tfidf_ask,tfidf_attack,tfidf_away,tfidf_back,tfidf_bad,tfidf_becaus,tfidf_becom,tfidf_befor,tfidf_believ,tfidf_best,tfidf_better,tfidf_bit,tfidf_bodi,tfidf_bpd,tfidf_brain,tfidf_call,tfidf_came,tfidf_care,tfidf_caus,tfidf_chang,tfidf_come,tfidf_complet,tfidf_constant,tfidf_control,tfidf_could,tfidf_coupl,tfidf_cri,tfidf_day,tfidf_deal,tfidf_depress,tfidf_diagnos,tfidf_die,tfidf_differ,tfidf_disord,tfidf_doctor,tfidf_doe,tfidf_done,tfidf_dont,tfidf_drink,tfidf_drug,tfidf_eat,tfidf_els,tfidf_emot,tfidf_end,tfidf_enough,tfidf_etc,tfidf_even,tfidf_ever,tfidf_everi,tfidf_everyon,tfidf_everyth,tfidf_experi,tfidf_famili,tfidf_fear,tfidf_feel,tfidf_feel like,tfidf_felt,tfidf_final,tfidf_find,tfidf_first,tfidf_food,tfidf_found,tfidf_friend,tfidf_fuck,tfidf_get,tfidf_give,tfidf_go,tfidf_good,tfidf_got,tfidf_great,tfidf_guess,tfidf_guy,tfidf_happen,tfidf_happi,tfidf_hard,tfidf_hate,tfidf_head,tfidf_health,tfidf_hear,tfidf_heart,tfidf_help,tfidf_high,tfidf_home,tfidf_hope,tfidf_hour,tfidf_hous,tfidf_hurt,tfidf_idea,tfidf_im,tfidf_issu,tfidf_job,tfidf_keep,tfidf_kill,tfidf_kind,tfidf_know,tfidf_last,tfidf_late,tfidf_leav,tfidf_left,tfidf_let,tfidf_life,tfidf_like,tfidf_littl,tfidf_live,tfidf_long,tfidf_look,tfidf_lose,tfidf_lost,tfidf_lot,tfidf_love,tfidf_made,tfidf_make,tfidf_mani,tfidf_mayb,tfidf_mean,tfidf_med,tfidf_medic,tfidf_mental,tfidf_might,tfidf_mind,tfidf_mom,tfidf_month,tfidf_move,tfidf_much,tfidf_need,tfidf_never,tfidf_new,tfidf_next,tfidf_night,tfidf_normal,tfidf_noth,tfidf_notic,tfidf_old,tfidf_onc,tfidf_one,tfidf_onli,tfidf_pain,tfidf_panic,tfidf_parent,tfidf_part,tfidf_past,tfidf_peopl,tfidf_person,tfidf_place,tfidf_pleas,tfidf_point,tfidf_possibl,tfidf_post,tfidf_pretti,tfidf_probabl,tfidf_problem,tfidf_ptsd,tfidf_put,tfidf_question,tfidf_quit,tfidf_read,tfidf_real,tfidf_realli,tfidf_reason,tfidf_recent,tfidf_relationship,tfidf_rememb,tfidf_right,tfidf_said,tfidf_say,tfidf_scare,tfidf_school,tfidf_see,tfidf_seem,tfidf_self,tfidf_sever,tfidf_shit,tfidf_sinc,tfidf_situat,tfidf_sleep,tfidf_social,tfidf_someon,tfidf_someth,tfidf_sometim,tfidf_sorri,tfidf_start,tfidf_stay,tfidf_still,tfidf_stop,tfidf_stress,tfidf_struggl,tfidf_stuff,tfidf_suicid,tfidf_support,tfidf_sure,tfidf_symptom,tfidf_take,tfidf_talk,tfidf_tell,tfidf_thank,tfidf_therapi,tfidf

In [48]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

df['post'] = df.iloc[ :,3].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ryan_\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [50]:
print(df.iloc[4,3])

unable stop talking ideas experiences hey know come find particularly times anxiety cannot stop saying things loud even thoughts necessarily parts conversations random sentences etc sound insane home holidays starting confuse family know diagnosis experiences anything particular helped
